In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Load the Dataset
df = pd.read_csv("Iris.csv")
X = df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
y = df['Species']

# 2. Normalize the Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# 4. Experiment with Different Values of K
k_values = range(1, 16)
accuracies = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"K={k}: Accuracy={acc:.4f}")

# 5. Plot Accuracy for Different K
plt.figure(figsize=(8, 5))
plt.plot(k_values, accuracies, marker='o')
plt.xlabel("Number of Neighbors (K)")
plt.ylabel("Accuracy")
plt.title("KNN Accuracy for Different Values of K")
plt.grid(True)
plt.show()

# 6. Use the Best K for Final Model
best_k = k_values[np.argmax(accuracies)]
print(f"\nBest K: {best_k} with accuracy {max(accuracies):.4f}")
knn_final = KNeighborsClassifier(n_neighbors=best_k)
knn_final.fit(X_train, y_train)
y_pred_final = knn_final.predict(X_test)

# 7. Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_final))
print("Classification Report:\n", classification_report(y_test, y_pred_final))
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_final):.4f}")

# 8. Optional: Visualize Decision Boundaries (only for first two features)
def plot_decision_boundaries(X, y, model, title):
    from matplotlib.colors import ListedColormap
    X_plot = X[:, :2]  # We only use first two features for visualization
    x_min, x_max = X_plot[:, 0].min() - 1, X_plot[:, 0].max() + 1
    y_min, y_max = X_plot[:, 1].min() - 1, X_plot[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel(), 
                            np.zeros_like(xx.ravel()), 
                            np.zeros_like(xx.ravel())])
    Z = pd.Categorical(Z).codes
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.2, cmap=ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF']))
    for i, class_label in enumerate(np.unique(y)):
        plt.scatter(X_plot[y == class_label, 0], X_plot[y == class_label, 1], 
                    label=class_label, edgecolor='k')
    plt.xlabel('SepalLengthCm (scaled)')
    plt.ylabel('SepalWidthCm (scaled)')
    plt.title(title)
    plt.legend()
    plt.show()

# Visualize with best K on first two features
knn_2d = KNeighborsClassifier(n_neighbors=best_k)
knn_2d.fit(X_train[:, :2], y_train)
plot_decision_boundaries(X_test, y_test, knn_2d, f"KNN Decision Boundary (K={best_k})")
